In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import numpy as np

from tensorflow.keras.models import load_model

In [23]:
## load the trained model, scaler and encoder
model = load_model('churn_model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_enocder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('onehot_encoder_geography.pkl', 'rb') as f:
    label_encoder_geo = pickle.load(f)


In [24]:
data  = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [34]:
input_data = {
    'CreditScore': 619,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 42,
    'Tenure': 2,
    'Balance': 0.00,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348.88,
    'Exited': 1
}

In [46]:
# Convert input_data dictionary to DataFrame
input_df = pd.DataFrame([input_data])

# Apply label encoding on the "Gender" column
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

# Apply one-hot encoding on the "Geography" column
encoded_geography = label_encoder_geo.transform(input_df[['Geography']])
# Convert the encoded geography column to a dataframe
encoded_geography_df = pd.DataFrame(encoded_geography.toarray(), columns=label_encoder_geo.get_feature_names_out(['Geography']))
input_df = pd.concat([input_df, encoded_geography_df], axis=1)

#drop geography column
input_df.drop(['Geography','Exited'], axis=1, inplace=True)

# Scale the input_df using the trained scaler
input_df_scaled = scaler.transform(input_df)
input_df_scaled


array([[-0.33880827,  0.91324755,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [47]:
## Predict Churn
prediction = model.predict(input_df_scaled)
prediction

1/1 [==============================] - 2s 2s/step


array([[0.37057626]], dtype=float32)

In [48]:
prediction_probability = prediction[0][0]
prediction_probability

0.37057626

In [49]:
if prediction_probability > 0.5:
    print("Customer will churn")
else:
    print("Customer will not churn")

Customer will not churn
